- 共享变量
    - 当多个线程同时访问一个变量的时候,会产生共享变量的问题
    

In [1]:
import threading

sum = 0
loopSun = 1000000

# 从0加到1000000
def myAdd():
    global sum,loopSun
    for i in range(loopSun):
        sum+= 1

# 从1000000减到0
def myMinu():
    global sum, loopSun
    for i in range(loopSun):
        sum -= 1
def main():
    t1 = threading.Thread(target=myAdd, args=())
    t2 = threading.Thread(target=myMinu, args=())
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
if __name__ == '__main__': 
    main()
    print(sum)
    
# 我们预期结果应该是0 但是每次执行结果都不一样 这就是共享变量问题

-22619


- 解决变量问题: 锁 , 信号灯
- 锁(Lock):
    - 是一个标志, 表示一个线程在占用一些资源
    - 使用方法
        - 上锁
        - 使用共享资源, 放心使用
        - 取消锁, 释放锁

In [2]:
import threading

sum = 0
loopSum = 1000000

lock = threading.Lock()

def myAdd():
    global sum , loopSum
    for i in range(loopSum):
        # 上锁, 申请锁
        lock.acquire()
        sum += 1
        # 释放锁
        lock.release()
    
def myMinu():
    global sum, loopSum
    for i in range(loopSum):
        # 上锁
        lock.acquire()
        sum -= 1
        # 释放锁
        lock.release()
        
if __name__ == '__main__':
    t1 = threading.Thread(target=myAdd, args=())
    t2 = threading.Thread(target=myMinu, args=())
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
    print(sum)
# 此时就不会产生变量问题       

0


- 线程安全问题:
    - 如果一个资源/变量, 他对于多线程来讲, 不用加锁也不会引起任何问题, 则称为线程安全
    - 线程安全变量: queue
    - 线程不安全变量: list, set, dict
- 生产者消费者问题:
    - 一个模型,可以用来搭建消息队列
    - queue是一个用来存放变量的数据结构,特点是先进先出,内部元素排队, 可以理解成一个特殊的list
    

In [ ]:
import threading
import time
import queue

class Producer(threading.Thread):
    def run(self):
        global queue
        count = 0
        while True:
            # qsize返回queue内容长度
            if queue.qsize() < 1000:
                for i in range(100):
                    count = count +1
                    msg = '生成产品'+str(count)
                    # put是往queue中放入消息
                    queue.put(msg)
                    print(msg)                 
            time.sleep(0.5)
# 以上代码 当queue的长度小于1000的时候,每0.5秒生产(放入)一个产品
                
class Consumer(threading.Thread):
    def run(self):
        global queue
        while True:
            if queue.qsize() > 100:
                for i in range(3):
                    # get是从queue中取出一个值
                    msg = self.name + '消费了' + queue.get()
                    print(msg)
            time.sleep(1)
# 以上代码 当queue的长度大于100的时候,每1秒消费(取出)一个产品
            
if __name__ == '__main__':
    queue = queue.Queue()
    
    for i in range(500):
        queue.put('初始产品'+str(i))
        
    for i in range(2):
        p = Producer()
        p.start()
        
    for i in range(5):
        c = Consumer()
        c.start()
# 以上代码 往queue中放入500个初始化产品
# 生成两个Producer和生成5个Consumer

- 死锁问题

In [ ]:
import threading
import time

lock_1 = threading.Lock()
lock_2 = threading.Lock()

def func_1():
    print("func_1 starting...")
    
    lock_1.acquire()
    print("func_1 申请了 lock_1")
    time.sleep(2)
    
    lock_2.acquire()
    print("func_1 申请了 lock_2")

    lock_2.release()
    print("func_1 释放了 lock_2")
    
    lock_1.release()
    print("func_1 释放了 lock_1")
    
    print("func_1 done...")
# 申请的顺序跟释放的顺序是相反的

def func_2():
    print("func_2 starting...")
    
    lock_2.acquire()
    print("func_2 申请了 lock_2")
    time.sleep(4)
    
    lock_1.acquire()
    print("func_2 申请了 lock_1")
    
    lock_1.release()
    print("func_2 释放了 lock_1")
    
    lock_2.release()
    print("func_2 释放了 lock_2")
    
    print("func_2 done...")
    
if __name__ == "__main__":
    print("主程序启动....")
    t1 = threading.Thread(target=func_1, args=())
    t2 = threading.Thread(target=func_2, args=())
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
 
'''
由于一开始fun_1申请了lock_1 和 func_2申请了lock_2  
两秒后func_1申请不到lock_2  四秒后fun_2也申请不到lock_1
所以成了死锁,程序死等
'''

主程序启动....
func_1 starting...
func_1 申请了 lock_1
func_2 starting...
func_2 申请了 lock_2


- 锁的等待时间问题

In [2]:
import time
import threading

lock_1 = threading.Lock()
lock_2 = threading.Lock()

def func_1():
    print("fun_1 starting...")
    
    lock_1.acquire(timeout=4)
    print("func_1 申请了 lock_1")
    time.sleep(2)
    
    rst = lock_2.acquire(timeout=2)
    if rst:
        prin("func_1 申请到了 lock_2")
        lock_2.release()
        print("func_1 释放了 lock_2")
    else:
        print("func_1 没有申请到 lcok_2")
    
    lock_1.release()
    print("fun_1 释放了 lock_1")
    
    print("func_1 done...")
    
def func_2():
    print("func_2 starting...")
    
    lock_2.acquire(timeout=4)
    print("func_2 申请了 lcok_2")
    time.sleep(4)
    
    rst = lock_1.acquire(timeout=2)
    if rst:
        print("func_2 申请到了 lock_1")
        lock_1.release()
        print("func_2 释放了 lock_1")
    else:
        print("func_2 没有申请到 lcok_1")
    
    lock_2.release()
    print("func_2 释放了 lcok_2")
    
    print("func_2 done...")
    
if __name__ == "__main__":
    print("主程序启动了...")
    
    t1 = threading.Thread(target=func_1, args=())
    t2 = threading.Thread(target=func_2, args=())
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()

# 此代码就有效解决了死锁问题

主程序启动了...
fun_1 starting...
func_1 申请了 lock_1
func_2 starting...
func_2 申请了 lcok_2
func_1 没有申请到 lcok_2
fun_1 释放了 lock_1
func_1 done...
func_2 申请到了 lock_1
func_2 释放了 lock_1
func_2 释放了 lcok_2
func_2 done...


- semaphore
    - 允许一个资源最多有几个线程同时使用
    

In [11]:
import threading
import time

# 参数定义最多几个线程同时使用资源
semaphore = threading.Semaphore(2)

def func():
    if semaphore.acquire():
        # 打印2遍
        for i in range(2):
            # 获取当前线程的名称+ 
            print(' ' + threading.currentThread().getName() + ' get semaphore')
        time.sleep(3)
        semaphore.release()
        print(' ' + threading.currentThread().getName() + ' release semaphore')

# 执行5次func()
for i in range(5):
    t1 = threading.Thread(target=func, args=())
    t1.start()
    
# 以下运行结果显示 最多运行两个线程,当第一个启动的semaphore被释放后 才继续启动下一个线程

 Thread-56 get semaphore
 Thread-56 get semaphore
 Thread-57 get semaphore
 Thread-57 get semaphore
 Thread-56 release semaphore Thread-58 get semaphore
 Thread-58 get semaphore
 Thread-57 release semaphore
 Thread-59 get semaphore
 Thread-59 get semaphore

 Thread-58 release semaphore Thread-60 get semaphore Thread-59 release semaphore

 Thread-60 get semaphore

 Thread-60 release semaphore


- threading.Timer

In [17]:
# 参数定义 多少秒后执行什么函数

import time
import threading

def func():
    print("I am running")
    time.sleep(2)
    print("I am done")
    
if __name__ == "__main__":
    t = threading.Timer(3, func)
    t.start()
    
    for i in range(10):
        time.sleep(1)
        print(" {0}**************".format(i))
         

 0**************
 1**************
I am running 2**************

 3**************
I am done 4**************

 5**************
 6**************
 7**************
 8**************
 9**************


- 可重入锁
    - 一个锁, 可以被一个线程多次申请
    - 主要解决递归调用的时候,需要申请锁的情况
    

In [32]:
# 一般的锁 此代码执行后死锁
import time
import threading

num = 0
lock = threading.Lock()

class MyThread(threading.Thread):
    def run(self):
        global num 
        time.sleep(1)
        
        if lock.acquire(1):
            num += 1
            msg = self.name + 'set num to ' + str(num)
            print(msg)
            lock.acquire()
            
            lock.release()
            lock.release()

def func():
    for i in range(5):
        t = MyThread()
        t.start()
        
if __name__ == "__main__":
    func()

Thread-82set num to 1


In [34]:
# 用 可重入锁,就能执行以上代码
import time
import threading

num = 0
lock = threading.RLock()

class MyThread(threading.Thread):
    def run(self):
        global num 
        time.sleep(1)
        
        if lock.acquire():
            num += 1
            msg = self.name + 'set num to ' + str(num)
            print(msg)
            lock.acquire()

            lock.release()
            lock.release()
        
def func():
    for i in range(5):
        t = MyThread()
        t.start()
        
if __name__ == "__main__":
    func()

Thread-92set num to 1
Thread-95set num to 2
Thread-94set num to 3
Thread-93set num to 4
Thread-96set num to 5
